In [2]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 48.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=6055312184a059e4a8884ffffb0f96041d29f6eb94d94cf97085819ae6ae9a47
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [109]:
filename = "/content/predictive_maintenance.csv"
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv(filename, header=True, inferSchema=True, sep=',')
data.show()

+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure Type|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|     0|  No Failure|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|     0|  No Failure|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|     0|  No Failure|
|  4|    L47183|   L|              298.2|                  308.6|                  1433|       39.5|              7|     0|  No Failure|
|  5|    L47184|   L|              298.2|

In [110]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
#Convertimos las variables categoricas en numeros ficticios
indexer = StringIndexer(inputCol= "Failure Type", outputCol = "Failure")
indexed = indexer.fit(data).transform(data)


In [111]:
final = StringIndexer(inputCol="Type", outputCol="dos type")
df_final = final.fit(indexed).transform(indexed)
df_final.show()

+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+-------+--------+
|UDI|Product ID|Type|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure Type|Failure|dos type|
+---+----------+----+-------------------+-----------------------+----------------------+-----------+---------------+------+------------+-------+--------+
|  1|    M14860|   M|              298.1|                  308.6|                  1551|       42.8|              0|     0|  No Failure|    0.0|     1.0|
|  2|    L47181|   L|              298.2|                  308.7|                  1408|       46.3|              3|     0|  No Failure|    0.0|     0.0|
|  3|    L47182|   L|              298.1|                  308.5|                  1498|       49.4|              5|     0|  No Failure|    0.0|     0.0|
|  4|    L47183|   L|              298.2|                  308.6|           

In [112]:
#Eliminamos las columnas que no necesitamos

df_final2 = df_final.drop(*('Type', 'Failure Type','UDI','Product ID'))
df_final2.show()

+-------------------+-----------------------+----------------------+-----------+---------------+------+-------+--------+
|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure|dos type|
+-------------------+-----------------------+----------------------+-----------+---------------+------+-------+--------+
|              298.1|                  308.6|                  1551|       42.8|              0|     0|    0.0|     1.0|
|              298.2|                  308.7|                  1408|       46.3|              3|     0|    0.0|     0.0|
|              298.1|                  308.5|                  1498|       49.4|              5|     0|    0.0|     0.0|
|              298.2|                  308.6|                  1433|       39.5|              7|     0|    0.0|     0.0|
|              298.2|                  308.7|                  1408|       40.0|              9|     0|    0.0|     0.0|
|              298.1|           

In [113]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

df_final2.columns


['Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Target',
 'Failure',
 'dos type']

In [114]:
predictoras = VectorAssembler(inputCols=['Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]',
                                         'Target','Failure','dos type'],
                              outputCol='features')
predictoras


VectorAssembler_ca6e5daf6d07

In [115]:
output = predictoras.transform(df_final2)

In [116]:
output.show()

+-------------------+-----------------------+----------------------+-----------+---------------+------+-------+--------+--------------------+
|Air temperature [K]|Process temperature [K]|Rotational speed [rpm]|Torque [Nm]|Tool wear [min]|Target|Failure|dos type|            features|
+-------------------+-----------------------+----------------------+-----------+---------------+------+-------+--------+--------------------+
|              298.1|                  308.6|                  1551|       42.8|              0|     0|    0.0|     1.0|[308.6,1551.0,42....|
|              298.2|                  308.7|                  1408|       46.3|              3|     0|    0.0|     0.0|[308.7,1408.0,46....|
|              298.1|                  308.5|                  1498|       49.4|              5|     0|    0.0|     0.0|[308.5,1498.0,49....|
|              298.2|                  308.6|                  1433|       39.5|              7|     0|    0.0|     0.0|[308.6,1433.0,39....|
|     

In [117]:
output.select("features","Air temperature [K]").show(truncate=False)

+------------------------------------+-------------------+
|features                            |Air temperature [K]|
+------------------------------------+-------------------+
|[308.6,1551.0,42.8,0.0,0.0,0.0,1.0] |298.1              |
|[308.7,1408.0,46.3,3.0,0.0,0.0,0.0] |298.2              |
|[308.5,1498.0,49.4,5.0,0.0,0.0,0.0] |298.1              |
|[308.6,1433.0,39.5,7.0,0.0,0.0,0.0] |298.2              |
|[308.7,1408.0,40.0,9.0,0.0,0.0,0.0] |298.2              |
|[308.6,1425.0,41.9,11.0,0.0,0.0,1.0]|298.1              |
|[308.6,1558.0,42.4,14.0,0.0,0.0,0.0]|298.1              |
|[308.6,1527.0,40.2,16.0,0.0,0.0,0.0]|298.1              |
|[308.7,1667.0,28.6,18.0,0.0,0.0,1.0]|298.3              |
|[309.0,1741.0,28.0,21.0,0.0,0.0,1.0]|298.5              |
|[308.9,1782.0,23.9,24.0,0.0,0.0,2.0]|298.4              |
|[309.1,1423.0,44.3,29.0,0.0,0.0,2.0]|298.6              |
|[309.1,1339.0,51.1,34.0,0.0,0.0,1.0]|298.6              |
|[309.2,1742.0,30.0,37.0,0.0,0.0,1.0]|298.6             

In [118]:
data_final3 = output.select("features","Air temperature [K]")
data_final3.show(truncate=False)

+------------------------------------+-------------------+
|features                            |Air temperature [K]|
+------------------------------------+-------------------+
|[308.6,1551.0,42.8,0.0,0.0,0.0,1.0] |298.1              |
|[308.7,1408.0,46.3,3.0,0.0,0.0,0.0] |298.2              |
|[308.5,1498.0,49.4,5.0,0.0,0.0,0.0] |298.1              |
|[308.6,1433.0,39.5,7.0,0.0,0.0,0.0] |298.2              |
|[308.7,1408.0,40.0,9.0,0.0,0.0,0.0] |298.2              |
|[308.6,1425.0,41.9,11.0,0.0,0.0,1.0]|298.1              |
|[308.6,1558.0,42.4,14.0,0.0,0.0,0.0]|298.1              |
|[308.6,1527.0,40.2,16.0,0.0,0.0,0.0]|298.1              |
|[308.7,1667.0,28.6,18.0,0.0,0.0,1.0]|298.3              |
|[309.0,1741.0,28.0,21.0,0.0,0.0,1.0]|298.5              |
|[308.9,1782.0,23.9,24.0,0.0,0.0,2.0]|298.4              |
|[309.1,1423.0,44.3,29.0,0.0,0.0,2.0]|298.6              |
|[309.1,1339.0,51.1,34.0,0.0,0.0,1.0]|298.6              |
|[309.2,1742.0,30.0,37.0,0.0,0.0,1.0]|298.6             

In [119]:
train,test = data_final3.randomSplit([0.7,0.3])

In [120]:
from pyspark.ml.regression import LinearRegression

In [121]:
regresion = LinearRegression(featuresCol="features",
                             labelCol="Air temperature [K]")

In [122]:
train.model = regresion.fit(train)

In [123]:
train.model

LinearRegressionModel: uid=LinearRegression_1c70f65ad3ba, numFeatures=7

In [124]:
pred = train.model.evaluate(test)
pred.predictions.show(truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------------------------+-------------------+------------------+
|features                             |Air temperature [K]|prediction        |
+-------------------------------------+-------------------+------------------+
|(7,[0,1,2],[308.1,1662.0,31.3])      |297.2              |297.77496374393087|
|(7,[0,1,2],[308.1,1672.0,41.3])      |298.3              |297.7083140555999 |
|(7,[0,1,2],[308.4,1571.0,39.4])      |297.4              |298.09668542041675|
|(7,[0,1,2],[308.9,1398.0,51.5])      |298.8              |298.6467843568433 |
|(7,[0,1,2],[309.0,1408.0,47.2])      |299.2              |298.78983928947326|
|(7,[0,1,2],[309.0,1456.0,41.2])      |297.8              |298.817461876409  |
|(7,[0,1,2],[309.5,1461.0,49.9])      |300.7              |299.34867698820506|
|(7,[0,1,2],[310.1,1558.0,32.6])      |301.0              |300.14382720181453|
|(7,[0,1,2],[310.6,1455.0,43.9])      |301.6              |300.68304330580287|
|(7,[0,1,2],[310.6,1462.0,43.6])      |302.2        

In [125]:
#coefficient of the regression model
coeff = train.model.coefficients

#X and Y intercept
intr = train.model.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)

The coefficient of the model is : DenseVector([1.1767, -0.0002, -0.0064, -0.0, 1.1516, -0.2345, -0.005])
The Intercept of the model is : -64.186047


In [126]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="Air temperature [K]", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred.predictions)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred.predictions, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


RMSE: 0.953
MSE: 0.909
MAE: 0.804
r2: 0.778
